# 6.8610  PROJECT

## Install and import libraries

In [1]:
!pip install nltk
!pip install datasets
!pip install transformers[torch]
!pip install tokenizers
!pip install evaluate
!pip install rouge_score
!pip install sentencepiece
!pip install huggingface_hub
!pip install wandb

zsh:1: no matches found: transformers[torch]


In [2]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from datasets import Dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import os
import json
import torch
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import gc
import pandas as pd
import random
random.seed(42)

/Users/emilia/micromamba/envs/cs109b/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


## Load model

In [4]:
MODEL_NAME = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

/Users/emilia/micromamba/envs/cs109b/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


## Create datasets

### Create code dataset

In [5]:
def extract_text_before_example(text):
    return text.split("**Example 1:**")[0].strip()

In [6]:
def convert_folder_to_datasetdict_code(folder_path):
    with open(folder_path, 'r') as file:
        data = json.load(file)

    df = pd.json_normalize(data)
    dataset = Dataset.from_pandas(df[['code_with_problem', 'code_only']])
    dataset['question'] = dataset['code_with_problem'].apply(extract_text_before_example)
    return dataset

In [7]:
def convert_folder_to_datasetdict_code(folder_path):
    with open(folder_path, 'r') as file:
        data = json.load(file)

    df = pd.json_normalize(data)

    # Extract text before "Example 1" for the 'code_with_problem' column
    df['question'] = df['code_with_problem'].apply(extract_text_before_example)

    # Rename columns and create the dataset
    df.rename(columns={'code_only': 'answer'}, inplace=True)
    dataset = Dataset.from_pandas(df[['question', 'answer']])

    return dataset

In [8]:
code_dataset = convert_folder_to_datasetdict_code('data/code/leetcode-solutions.json')
train_dataset, test_dataset = train_test_split(code_dataset, test_size=0.2)
train_dataset = Dataset.from_dict(train_dataset)
test_dataset = Dataset.from_dict(test_dataset)

/Users/emilia/micromamba/envs/cs109b/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [9]:
code_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print(code_dict)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1887
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 472
    })
})


In [10]:
n_train = code_dict['train'].num_rows
n_test = code_dict['test'].num_rows
n_total = n_train + n_test

In [11]:
code_dict["train"][0]

{'question': '# You are given an array `points` where `points[i] = [xi, yi]` represents a point on an **X-Y** plane.\n\n**Straight lines** are going to be added to the **X-Y** plane, such that every point is covered by at **least** one line.\n\nReturn _the **minimum** number of **straight lines** needed to cover all the points_.',
 'answer': '```python\nfrom math import gcd\nfrom itertools import combinations\n\ndef minStraightLines(points):\n    lines = set()\n    for p1, p2 in combinations(points, 2):\n        dy, dx = p2[1] - p1[1], p2[0] - p1[0]\n        g = gcd(dx, dy)\n        lines.add((dy // g, dx // g))\n    return len(lines)\n```\n\n\n'}

### Create general knowledge dataset

In [12]:
csv_file_path = 'data/general/general.csv'
general = pd.read_csv(csv_file_path)
general = general.sample(n=n_total, random_state=42)
general['id'] = range(len(general))
train_sample, test_sample = train_test_split(general, test_size=0.2, random_state=42)
train_sample.reset_index(drop=True, inplace=True)
test_sample.reset_index(drop=True, inplace=True)

train_dataset = Dataset.from_pandas(train_sample[['question', 'answer']])
test_dataset = Dataset.from_pandas(test_sample[['question', 'answer']])

general_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
})

print(general_dict)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1887
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 472
    })
})


/Users/emilia/micromamba/envs/cs109b/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/Users/emilia/micromamba/envs/cs109b/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [13]:
general_dict["train"][0]

{'question': 'Add 1 letter to "Iowa" to get the name of this tribe who lived south of the Iowa',
 'answer': 'Kiowa'}

### Create math dataset

In [14]:
def convert_folder_to_datasetdict_math(folder_path):
    #data = {"id": [], "question": [], "level": [], "type": [], "answer": []}
    data = {"question": [], "answer": []}

    subject_dictionary = {
        "algebra": 1,
        "counting_and_probability": 2,
        "geometry": 3,
        "intermediate_algebra": 4,
        "number_theory": 5,
        "prealgebra": 6,
        "precalculus": 7
    }

    for subdir, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(subdir, file)
            folder_name = os.path.basename(os.path.normpath(subdir))
            
            with open(file_path, "r") as f:
                json_data = json.load(f)
                problem = json_data.get("problem", "")
                level = json_data.get("level", "")
                type_ = json_data.get("type", "")
                solution = json_data.get("solution", "")

                # Generate id from subject code and file name
                file_name = os.path.splitext(file)[0]
                subject_code = subject_dictionary.get(folder_name, 0)  # Default to 0 if not found
                id_ = f"{subject_code}_{file_name}"

                #data["id"].append(id_)
                data["question"].append(problem)
                #data["level"].append(level)
                #data["type"].append(type_)
                data["answer"].append(solution)

    dataset = Dataset.from_dict(data)
    return dataset

In [15]:
math_train = convert_folder_to_datasetdict_math("data/math/train/")
math_test = convert_folder_to_datasetdict_math("data/math/test/")

In [16]:
math_train = math_train.shuffle(seed=42)
math_train = math_train.select(range(n_train))

math_test = math_test.shuffle(seed=42)
math_test = math_test.select(range(n_test))

math_dict = DatasetDict({
    'train': math_train,
    'test': math_test
})

print(math_dict)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1887
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 472
    })
})


In [17]:
math_dict["train"][0]

{'question': 'Three positive integers $a$, $b$, and $c$ satisfy $a\\cdot b\\cdot c=8!$ and $a<b<c$.  What is the smallest possible value of $c-a$?',
 'answer': 'Our goal is to divide the factors of 8! into three groups in such a way that the products of the factors in each group are as close together as possible.  Write $8!$ as $8\\cdot 7 \\cdot 6 \\cdot 5\\cdot 4\\cdot 3\\cdot 2$.  Observe that $30^3<8!<40^3$, so the cube root of $8!$ is between $30$ and $40$.  With this in mind, we group $7$ and $5$ to make one factor of $35$.  We can also make a factor of $36$ by using $6$ along with $3$ and $2$.  This leaves $8$ and $4$ which multiply to give $32$.  The assignment $(a,b,c)=(32,35,36)$ has the minimum value of $c-a$, since $31$, $33$, $34$, $37$, $38$, and $39$ contain prime factors not present in $8!$.  Therefore, the minimum value of $c-a$ is $\\boxed{4}$.'}

### Create 50% samples

In [18]:
math_train_sample = math_dict['train'].shuffle(seed=42).select([i for i in range(math_dict['train'].num_rows // 2)])
math_test_sample = math_dict['test'].shuffle(seed=42).select([i for i in range(math_dict['test'].num_rows // 2)])

general_train_sample = general_dict['train'].shuffle(seed=42).select([i for i in range(general_dict['train'].num_rows // 2+1)])
general_test_sample = general_dict['test'].shuffle(seed=42).select([i for i in range(general_dict['test'].num_rows // 2)])

code_train_sample = code_dict['train'].shuffle(seed=42).select([i for i in range(code_dict['train'].num_rows // 2)])
code_test_sample = code_dict['test'].shuffle(seed=42).select([i for i in range(code_dict['test'].num_rows // 2)])

### Create 50% general 50% math dataset


In [19]:
general_math_dict = DatasetDict({
    'train': Dataset.from_dict({
        'question': math_train_sample['question'] + general_train_sample['question'],
        'answer': math_train_sample['answer'] + general_train_sample['answer'],
    }),
    'test': Dataset.from_dict({
        'question': math_test_sample['question'] + general_test_sample['question'],
        'answer': math_test_sample['answer'] + general_test_sample['answer'],
    })
})


general_math_dict = DatasetDict({
    'train': general_math_dict['train'].shuffle(seed=42),
    'test': general_math_dict['test'].shuffle(seed=42)
})

In [20]:
general_math_dict["train"][0]

{'question': 'A 4-inch by 6-inch picture is enlarged for framing  by tripling its dimensions.  A 2-inch-wide border  is then placed around each side of the enlarged  picture, as shown.  Thin metal framing is sold only  in increments of one foot.  What is the minimum  number of linear feet of framing that must be  purchased to go around the perimeter of the border?\n\n[asy]\n\ndraw((0,0)--(14,0)--(14,20)--(0,20)--cycle,linewidth(2));\n\ndraw((4,4)--(10,4)--(10,16)--(4,16)--cycle);\n\nlabel("border",(7,17),N);\n\nlabel("picture",(7,8),N);\n\nlabel("frame",(14,5),E);\n\ndraw((17.5,7.5)--(14.5,7.5),Arrow);\ndraw((10.5,7.5)--(13.5,7.5),Arrow);\n\n[/asy]',
 'answer': 'After the picture is enlarged by tripling its dimensions, the dimensions become $12\\times18$. After the border is added, the dimensions of the picture increase to $16\\times22$ (since each side has a 2-inch border). The perimeter is $16+16+22+22=76$ inches. Since $76/12=6\\frac{1}{3}$, we need $\\boxed{7}$ feet of framing to g

### Create 50% general 50% code dataset


In [21]:
general_code_dict = DatasetDict({
    'train': Dataset.from_dict({
        'question': code_train_sample['question'] + general_train_sample['question'],
        'answer': code_train_sample['answer'] + general_train_sample['answer'],
    }),
    'test': Dataset.from_dict({
        'question': code_test_sample['question'] + general_test_sample['question'],
        'answer': code_test_sample['answer'] + general_test_sample['answer'],
    })
})

general_code_dict = DatasetDict({
    'train': general_code_dict['train'].shuffle(seed=42),
    'test': general_code_dict['test'].shuffle(seed=42)
})

print(general_code_dict)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1887
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 472
    })
})


In [22]:
general_code_dict["train"][0]

{'question': '# You are given an array `nums`. You can rotate it by a non-negative integer `k` so that the array becomes `[nums[k], nums[k + 1], ... nums[nums.length - 1], nums[0], nums[1], ..., nums[k-1]]`. Afterward, any entries that are less than or equal to their index are worth one point.\n\n*   For example, if we have `nums = [2,4,1,3,0]`, and we rotate by `k = 2`, it becomes `[1,3,0,2,4]`. This is worth `3` points because `1 > 0` \\[no points\\], `3 > 1` \\[no points\\], `0 <= 2` \\[one point\\], `2 <= 3` \\[one point\\], `4 <= 4` \\[one point\\].\n\nReturn _the rotation index_ `k` _that corresponds to the highest score we can achieve if we rotated_ `nums` _by it_. If there are multiple answers, return the smallest such index `k`.',
 'answer': '```python\ndef moves_to_chessboard(board):\n    N = len(board)\n    row, col = 0, 0\n    row_count, col_count = 0, 0\n\n    for i in range(N):\n        for j in range(N):\n            if (board[0][0] ^ board[i][0] ^ board[0][j] ^ board[i]

## Different Sizes

In [59]:
def smaller_df(train, test, num_train = 500, seed = 42):
    num_test = int(((100*num_train)/80) - num_train) #to get 20% test

    train_small = train.shuffle(seed = seed).select(range(num_train))
    test_small = test.shuffle(seed = seed).select(range(num_test))
    
    dict_final =  DatasetDict({
    'train': train_small,
    'test': test_small
})
    
    return dict_final, train_small, test_small
                                                    

In [60]:
### General Knwoledge/Facts
general_dict_small, general_train_small, general_test_small = smaller_df(general_dict['train'],
                                                                         general_dict['test'])

##Math
math_dict_small, math_train_small, math_test_small = smaller_df(math_train, math_test)

### Code
code_dict_small, code_train_small, code_test_small = smaller_df(code_dict['train'], code_dict['test'])

## Preprocessing

In [65]:
prefix = "Please answer this question: "

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
  
   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["answer"], 
                      max_length=512,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [66]:
tokenized_dataset_math = math_dict.map(preprocess_function, batched=True)
tokenized_dataset_code = code_dict.map(preprocess_function, batched=True)
tokenized_dataset_general = general_dict.map(preprocess_function, batched=True)
tokenized_dataset_general_code = general_code_dict.map(preprocess_function, batched=True)
tokenized_dataset_general_math = general_math_dict.map(preprocess_function, batched=True)

### Smaller Models
tokenized_dataset_math_small = math_dict_small.map(preprocess_function, batched=True)
tokenized_dataset_code_small = code_dict_small.map(preprocess_function, batched=True)
tokenized_dataset_general_small = general_dict_small.map(preprocess_function, batched=True)


In [67]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [68]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

## Fine-tuning

### Experiment

In [69]:
def training_args(output_dir, L_RATE = 3e-4, BATCH_SIZE = 4, PER_DEVICE_EVAL_BATCH = 4, WEIGHT_DECAY = 0.01, SAVE_TOTAL_LIM = 3, NUM_EPOCHS = 3, OVERWRITE_OUTPUT_DIR = True, LOAD_BEST_MODEL_AT_END = True):
    training_args = Seq2SeqTrainingArguments(
       output_dir=output_dir,
       overwrite_output_dir=OVERWRITE_OUTPUT_DIR,
       save_strategy="epoch",
       evaluation_strategy="epoch",
       load_best_model_at_end=LOAD_BEST_MODEL_AT_END,
       learning_rate=L_RATE,
       per_device_train_batch_size=BATCH_SIZE,
       per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
       weight_decay=WEIGHT_DECAY,
       save_total_limit=SAVE_TOTAL_LIM,
       num_train_epochs=NUM_EPOCHS,
       predict_with_generate=True,
       push_to_hub=False
    )
    return training_args

In [70]:
lrs = [3e-3, 3e-4, 3e-5] 

### Math

In [48]:
for lr in lrs:
    model_math = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_math = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_math)
    
    output_dir_root = "./results/math"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_math = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_math = Seq2SeqTrainer(
       model=model_math,
       args=training_args_math,
       train_dataset=tokenized_dataset_math["train"], 
       eval_dataset=tokenized_dataset_math["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_math,
       compute_metrics=compute_metrics
    )

    trainer_math.train()

    del model_math
    gc.collect()
    torch.cuda.empty_cache()

### Code

In [49]:
for lr in lrs:
    model_code = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_code = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_code)
    
    output_dir_root = "./results/code"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_code = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_code = Seq2SeqTrainer(
       model=model_code,
       args=training_args_code,
       train_dataset=tokenized_dataset_code["train"], 
       eval_dataset=tokenized_dataset_code["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_code,
       compute_metrics=compute_metrics
    )

    trainer_code.train()

    del model_code
    gc.collect()
    torch.cuda.empty_cache()

### General

In [50]:
for lr in lrs:
    model_general = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_general = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_general)

    output_dir_root = "./results/general"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_general = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_general = Seq2SeqTrainer(
       model=model_general,
       args=training_args_general,
       train_dataset=tokenized_dataset_general["train"], 
       eval_dataset=tokenized_dataset_general["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_general,
       compute_metrics=compute_metrics
    )

    trainer_general.train()

    del model_general
    gc.collect()
    torch.cuda.empty_cache()

### General + Code

In [51]:
for lr in lrs:
    model_general_code = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_general_code = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_general_code)

    output_dir_root = "./results/general_code"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_general_code = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_general_code = Seq2SeqTrainer(
       model=model_general_code,
       args=training_args_general_code,
       train_dataset=tokenized_dataset_general_code["train"], 
       eval_dataset=tokenized_dataset_general_code["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_general_code,
       compute_metrics=compute_metrics
    )

    trainer_general_code.train()

    del model_general_code
    gc.collect()
    torch.cuda.empty_cache()

### General + Math

In [52]:
for lr in lrs:
    model_general_math = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_general_math = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_general_math)

    output_dir_root = "./results/general_math"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_general_math = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_general_math = Seq2SeqTrainer(
       model=model_general_math,
       args=training_args_general_math,
       train_dataset=tokenized_dataset_general_math["train"], 
       eval_dataset=tokenized_dataset_general_math["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_general_math,
       compute_metrics=compute_metrics
    )

    trainer_general_math.train()

    del model_general_math
    gc.collect()
    torch.cuda.empty_cache()

### Extension

#### 1. Small Math

In [ ]:
for lr in lrs:
    model_small_math = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_math_small = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_small_math)
    
    output_dir_root = "./results/small_math"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_math_small = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_math_small = Seq2SeqTrainer(
       model=model_small_math,
       args=training_args_math_small,
       train_dataset=tokenized_dataset_math_small["train"], 
       eval_dataset=tokenized_dataset_math_small["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_math_small,
       compute_metrics=compute_metrics
    )

    trainer_math_small.train()

    del model_small_math
    gc.collect()
    torch.cuda.empty_cache()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: answer, question. If answer, question are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/Users/emilia/micromamba/envs/cs109b/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 500
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 375
  Number of trainable parameters = 222903552
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wa

  ········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /Users/emilia/.netrc


Epoch,Training Loss,Validation Loss


### 2. Code Small

In [ ]:
for lr in lrs:
    model_small_code = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_code_small = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_small_code)
    
    output_dir_root = "./results/small_code"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_code_small = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_code_small = Seq2SeqTrainer(
       model=model_small_code,
       args=training_args_code_small,
       train_dataset=tokenized_dataset_code_small["train"], 
       eval_dataset=tokenized_dataset_code_small["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_code_small,
       compute_metrics=compute_metrics
    )

    trainer_code_small.train()

    del model_small_code
    gc.collect()
    torch.cuda.empty_cache()

### 3. General Knowledge

In [ ]:
for lr in lrs:
    model_small_general = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_general_small = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_small_general)

    output_dir_root = "./results/small_general"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_general_small = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_general_small = Seq2SeqTrainer(
       model=model_small_general,
       args=training_args_general_small,
       train_dataset=tokenized_dataset_general_small["train"], 
       eval_dataset=tokenized_dataset_general_small["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_general_small,
       compute_metrics=compute_metrics
    )

    trainer_general_small.train()

    del model_small_general
    gc.collect()
    torch.cuda.empty_cache()